In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "svg"

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta

from typing import List, Union

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score



In [6]:
import os
os.getcwd()

'/Users/nelsonpeace/Projects/AFML_various_repos_public/Adv_Fin_ML_public'

In [7]:
import nbimporter
from Functions import getTEvents, get_dollar_bars, get_returns

## Auxilary functions

In [3]:
def get_daily_vol(close: pd.Series, span0: int = 20) -> pd.Series:
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = pd.Series(close.index[df0 - 1], index=close.index[close.shape[0] - df0.shape[0]:])
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1    # daily returns
    df0 = df0.ewm(span=span0).std()
    return df0

In [4]:
def apply_tripple_barrier(close: pd.Series, events: pd.DataFrame,
                                   pt_sl: List, molecule: np.ndarray) -> pd.DataFrame:
    '''
    Labeling observations using tripple-barrier method
    
        Parameters:
            close (pd.Series): close prices of bars
            events (pd.DataFrame): dataframe with columns:
                                   - t1: The timestamp of vertical barrier (if np.nan, there will not be
                                         a vertical barrier)
                                   - trgt: The unit width of the horizontal barriers
            pt_sl (list): list of two non-negative float values:
                          - pt_sl[0]: The factor that multiplies trgt to set the width of the upper barrier.
                                      If 0, there will not be an upper barrier.
                          - pt_sl[1]: The factor that multiplies trgt to set the width of the lower barrier.
                                      If 0, there will not be a lower barrier.
            molecule (np.ndarray):  subset of event indices that will be processed by a
                                    single thread (will be used later)
        
        Returns:
            out (pd.DataFrame): dataframe with columns [pt, sl, t1] corresponding to timestamps at which
                                each barrier was touched (if it happened)
    '''
    events_ = events.loc[molecule]
    out = events_[['t1']].copy(deep=True)
    if pt_sl[0] > 0:
        pt = pt_sl[0] * events_['trgt']
    else:
        pt = pd.Series(data=[np.nan] * len(events.index), index=events.index)    # NaNs
    if pt_sl[1] > 0:
        sl = -pt_sl[1] * events_['trgt']
    else:
        sl = pd.Series(data=[np.nan] * len(events.index), index=events.index)    # NaNs
    
    for loc, t1 in events_['t1'].fillna(close.index[-1]).iteritems():
        df0 = close[loc: t1]                                       # path prices
        df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']     # path returns
        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()        # earlisest stop loss
        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()        # earlisest profit taking
    return out

In [5]:
# including metalabeleing possibility
def get_events_tripple_barrier(
    close: pd.Series, tEvents: np.ndarray, pt_sl: float, trgt: pd.Series, minRet: float,
    numThreads: int = 1, t1: Union[pd.Series, bool] = False, side: pd.Series = None
) -> pd.DataFrame:
    '''
    Getting times of the first barrier touch
    
        Parameters:
            close (pd.Series): close prices of bars
            tEvents (np.ndarray): np.ndarray of timestamps that seed every barrier (they can be generated
                                  by CUSUM filter for example)
            pt_sl (float): non-negative float that sets the width of the two barriers (if 0 then no barrier)
            trgt (pd.Series): s series of targets expressed in terms of absolute returns
            minRet (float): minimum target return required for running a triple barrier search
            numThreads (int): number of threads to use concurrently
            t1 (pd.Series): series with the timestamps of the vertical barriers (pass False
                            to disable vertical barriers)
            side (pd.Series) (optional): metalabels containing sides of bets
        
        Returns:
            events (pd.DataFrame): dataframe with columns:
                                       - t1: timestamp of the first barrier touch
                                       - trgt: target that was used to generate the horizontal barriers
                                       - side (optional): side of bets
    '''
    trgt = trgt.loc[trgt.index.intersection(tEvents)]
    trgt = trgt[trgt > minRet]
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)
    if side is None:
        side_, pt_sl_ = pd.Series(np.array([1.] * len(trgt.index)), index=trgt.index), [pt_sl[0], pt_sl[0]]
    else:
        side_, pt_sl_ = side.loc[trgt.index.intersection(side.index)], pt_sl[:2]
    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])
    df0 = apply_tripple_barrier(close, events, pt_sl_, events.index)
#     df0 = mpPandasObj(func=apply_tripple_barrier, pdObj=('molecule', events.index),
#                       numThreads=numThreads, close=close, events=events, pt_sl=[pt_sl, pt_sl])
    events['t1'] = df0.dropna(how='all').min(axis=1)
    if side is None:
        events = events.drop('side', axis=1)
    return events

Here we don't use the `mpPandasObj` multiprocessing function that appears in the book later in Chapter 20.

In [ ]:
def add_vertical_barrier(close: pd.Series, tEvents: np.ndarray, numDays: int) -> pd.Series:
    t1 = close.index.searchsorted(tEvents + pd.Timedelta(days=numDays))
    t1 = t1[t1 < close.shape[0]]
    t1 = pd.Series(close.index[t1], index=tEvents[:t1.shape[0]])    # adding NaNs to the end
    return t1

In [ ]:
# including metalabeling possibility & modified to generate 0 labels
def get_bins(close: pd.Series, events: pd.DataFrame, t1: Union[pd.Series, bool] = False) -> pd.DataFrame:
    '''
    Generating labels with possibility of knowing the side (metalabeling)
    
        Parameters:
            close (pd.Series): close prices of bars
            events (pd.DataFrame): dataframe returned by 'get_events' with columns:
                                   - index: event starttime
                                   - t1: event endtime
                                   - trgt: event target
                                   - side (optional): position side
            t1 (pd.Series): series with the timestamps of the vertical barriers (pass False
                            to disable vertical barriers)
        
        Returns:
            out (pd.DataFrame): dataframe with columns:
                                       - ret: return realized at the time of the first touched barrier
                                       - bin: if metalabeling ('side' in events), then {0, 1} (take the bet or pass)
                                              if no metalabeling, then {-1, 1} (buy or sell)
    '''
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[events_.index] - 1
    if 'side' in events_:
        out['ret'] *= events_['side']
    out['bin'] = np.sign(out['ret'])
    if 'side' in events_:
        out.loc[out['ret'] <= 0, 'bin'] = 0
    else:
        if t1 is not None:
            vertical_first_touch_idx = events_[events_['t1'].isin(t1.values)].index
            out.loc[vertical_first_touch_idx, 'bin'] = 0
    return out

With metalabeling, using labels generated by `get_bins` function, ML algorithm is trained to make a purely binary decision - 0 or 1. When the predicted label is 1, we can use the probability of this secondary prediction to derive the size of the
bet, where the side (sign) of the position has been set by the primary (meta-) model.

In [ ]:
def drop_labels(labels: pd.DataFrame, min_pct: float = 0.05) -> pd.DataFrame:
    while True:
        df0 = labels['bin'].value_counts(normalize=True)
        if df0.min() > min_pct or df0.shape[0] < 3:
            break
        print('dropped label', df0.argmin(), df0.min())
        labels = labels[labels['bin'] != df0.index[df0.argmin()]]
    return labels

## Exercises

Here I use the clean dataset generated in `Chapter2.ipynb`. All the necessary functions implemented in that file are imported here.

In [ ]:
data = pd.read_csv('sp10-19.csv')
data.head()

### 1. Dollar Bars, CUSUM filter & Tripple-Barrier Method

In [8]:
# $50,000 per bar
dollar_bars = get_dollar_bars(data['price'].values, data['volume'].values, data['datetime'].values, 50000)
dollar_bars_df = pd.DataFrame(data=dollar_bars[:, 1:], index=dollar_bars[:, 0],
                           columns=['open', 'high', 'low', 'close', 'volume'])
dollar_bars_df = dollar_bars_df[~dollar_bars_df.index.duplicated(keep='first')]
# fig = go.Figure(
#     data=[go.Candlestick(x=dollar_bars_df.index, open=dollar_bars_df['open'],
#                          high=dollar_bars_df['high'], low=dollar_bars_df['low'], close=dollar_bars_df['close'])]
# ).update_layout(xaxis_title="Time", yaxis_title="Price", title="Dollar Bars")
# fig.show()

NameError: name 'data' is not defined

Aplly a symmetric CUSUM filter to obtained dollar bars with threshold equal to mean daily returns std:

In [ ]:
mean_std = get_daily_vol(dollar_bars_df['close']).mean()
dollar_ret = get_returns(dollar_bars)

tEvents = getTEvents(dollar_ret, h=mean_std)
chosen_bars_df = dollar_bars_df.loc[tEvents, :]

fig = go.Figure(
    go.Scatter(x=dollar_bars_df.index, y=dollar_bars_df['close'],
               line=dict(color="blue", width=1), name='Close Price')
).update_layout(xaxis_title="Time", yaxis_title="Price", title="Close Price and Selected Timestamps")
fig.add_trace(go.Scatter(
    x=chosen_bars_df.index, y=chosen_bars_df['close'], mode='markers', marker=dict(size=1), name='Selected Timestamps'
    )
)
fig.show()

Add vertical barriers:

In [ ]:
t1 = add_vertical_barrier(dollar_bars_df['close'], tEvents, numDays=1)
t1

Now let's apply the tripple-barrier method with `pt_sl=[1, 1]` and generated `t1`. Here we have to choose the minimum target return to run the tripple-barrier method, let it be `minRet=0.007` (so that I get more data points). As a `trgt` for horizontal barriers I use daily volatility.

In [ ]:
events = get_events_tripple_barrier(close=dollar_bars_df['close'], tEvents=tEvents, pt_sl=[1, 1],
                                    trgt=get_daily_vol(dollar_bars_df['close']), minRet=0.007,
                                    numThreads=1, t1=t1)
events

Finally, let's get out labels:

In [ ]:
labels = get_bins(close=dollar_bars_df['close'], events=events, t1=t1)
labels

In [ ]:
labels.describe()

Note that here we used the possibility of zero labels so there are 3 labels we get - 1, -1 and 0. Therefore, we can apply `drop_labels` to drop the least popular label:

In [ ]:
labels = drop_labels(labels, min_pct=0.05)
labels

No label is dropped meaning there is no huge imbalance in the `labels` dataset.

### 2. Crossing Moving Average Strategy

In [ ]:
dollar_bars_df_ma = dollar_bars_df.copy(deep=True)
dollar_bars_df_ma['ma'] = dollar_bars_df_ma['close'].rolling(30, min_periods=1).mean()

def get_upside_bars_ma(df: pd.DataFrame) -> pd.DataFrame:
    return df[(df['close'] < df['ma']) & (df.shift(-1)['close'] > df.shift(-1)['ma'])]

def get_downside_bars_ma(df: pd.DataFrame) -> np.ndarray:
    return df[(df['close'] > df['ma']) & (df.shift(-1)['close'] < df.shift(-1)['ma'])]

up_timestamps, down_timestamps = get_upside_bars_ma(dollar_bars_df_ma), get_downside_bars_ma(dollar_bars_df_ma)

fig = go.Figure(
    go.Scatter(x=dollar_bars_df_ma.index, y=dollar_bars_df_ma['close'], line=dict(color="blue", width=1),
               name='Close Price')
).update_layout(xaxis_title="Time", yaxis_title="Price", title="Close Price and MA")
fig.add_trace(go.Scatter(x=dollar_bars_df_ma.index, y=dollar_bars_df_ma['ma'], line=dict(color="purple", width=0.7),
                         name='MA20')
)
fig.add_trace(go.Scatter(
    x=up_timestamps.index, y=up_timestamps['close'], mode='markers',
    marker=dict(size=3, color='green', symbol=5), name='Upside Timestamps')
)
fig.add_trace(go.Scatter(
    x=down_timestamps.index, y=down_timestamps['close'], mode='markers',
    marker=dict(size=3, color='red', symbol=6), name='Downside Timestamps')
)
fig.show()

Now we have to derive metalabels using the events from the previous task (but now we have sides of bets):

In [ ]:
side_index = up_timestamps.index.union(down_timestamps.index)
side_data = []
for idx in side_index:
    if idx in up_timestamps.index:
        side_data.append(1)
    else:
        side_data.append(-1)
side = pd.Series(data=side_data, index=side_index)

events_ma = get_events_tripple_barrier(close=dollar_bars_df['close'], tEvents=tEvents, pt_sl=[1, 2],
                                       trgt=get_daily_vol(dollar_bars_df['close']), minRet=0.007,
                                       numThreads=1, t1=t1, side=side)
events_ma = events_ma.dropna()
events_ma

Now we have to get the bins corresponding to chosen events:

In [ ]:
bins_ma = get_bins(close=dollar_bars_df_ma['close'], events=events_ma, t1=t1)
bins_ma

Finally, we can train random forest to decide whether to trade or not. Here the only feature is the decision of the first-level algorithm - the side of the bet. In out case these are the sides given by the MA Crossing strategy.

In [ ]:
def print_results(rf: RandomForestClassifier, X_test: np.ndarray,
                  y_test: np.ndarray, y_pred: np.ndarray) -> None:
    print(f'RF accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'RF precision: {precision_score(y_test, y_pred)}')
    print(f'RF recall: {recall_score(y_test, y_pred)}')
    plot_roc_curve(rf, X_test, y_test)

X, y = events_ma['side'].values.reshape(-1, 1), bins_ma['bin'].values.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print_results(rf, X_test, y_test, y_pred)

### 3. Bollinger Bands Mean-Reverting Strategy

Here we do practically the same thing but use Bollinger bands as the primary model.

In [ ]:
def get_bollinger_bands(dollar_bars: pd.DataFrame, alpha: float) -> np.ndarray:
    prices = dollar_bars['close']                                    # taking close prices
    ma = (prices.rolling(30, min_periods=1).mean())                  # 30 bars moving average
    sigma = prices.rolling(30, min_periods=1).std()
    sigma[0] = 0
    b_upper, b_lower = (ma + alpha * sigma), (ma - alpha * sigma)    # bollinger bounds    
    return np.array([ma, b_upper, b_lower])

ma, b_upper, b_lower = get_bollinger_bands(dollar_bars_df, 1)
dollar_bars_df_bb = dollar_bars_df.copy(deep=True)
dollar_bars_df_bb['ma'], dollar_bars_df_bb['b_upper'], dollar_bars_df_bb['b_lower'] = ma, b_upper, b_lower

def get_upside_bars_bb(df: pd.DataFrame) -> pd.DataFrame:
    return df[(df['open'] < df['b_upper']) & (df['close'] > df['b_upper'])]

def get_downside_bars_bb(df: pd.DataFrame) -> np.ndarray:
    return df[(df['open'] > df['b_lower']) & (df['close'] < df['b_lower'])]

up_timestamps, down_timestamps = get_upside_bars_bb(dollar_bars_df_bb), get_downside_bars_bb(dollar_bars_df_bb)

fig = go.Figure(
    go.Scatter(x=dollar_bars_df_bb.index, y=dollar_bars_df_bb['close'], line=dict(color="blue", width=1),
               name='Close Price')
).update_layout(xaxis_title="Time", yaxis_title="Price", title="Close Price")
fig.add_trace(go.Scatter(
    x=up_timestamps.index, y=up_timestamps['close'], mode='markers',
    marker=dict(size=3, color='green', symbol=5), name='Upside Timestamps')
)
fig.add_trace(go.Scatter(
    x=down_timestamps.index, y=down_timestamps['close'], mode='markers',
    marker=dict(size=3, color='red', symbol=6), name='Downside Timestamps')
)
fig.show()

In [ ]:
side_index = up_timestamps.index.union(down_timestamps.index)
side_data = []
for idx in side_index:
    if idx in up_timestamps.index:
        side_data.append(1)
    else:
        side_data.append(-1)
side = pd.Series(data=side_data, index=side_index)

events_bb = get_events_tripple_barrier(close=dollar_bars_df['close'], tEvents=tEvents, pt_sl=[0, 2],
                                       trgt=get_daily_vol(dollar_bars_df['close']), minRet=0.007,
                                       numThreads=1, t1=t1, side=side)
events_bb = events_bb.dropna()
events_bb

In [ ]:
bins_bb = get_bins(close=dollar_bars_df_bb['close'], events=events_bb, t1=t1)
bins_bb

Now we can train the secondary random forest model. Apart from the sides given by the primary model, I add volatility and autocorreation to the features.

In [ ]:
X = pd.DataFrame(
    {'vol': get_daily_vol(dollar_bars_df['close']),
     'autocorr': dollar_ret.rolling(20, min_periods=1).corr(dollar_ret.shift(1)).dropna(),
     'side': events_bb['side']}
).dropna()
X = X[~X.index.duplicated(keep='first')]
y = bins_bb['bin'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print_results(rf, X_test, y_test, y_pred)

We can compare results of this 2-level model to results that can be achieved by using only primary model (based on Bollinger bands):

In [ ]:
# change classes from {-1, -1} to {0, 1}
y_pred_primary = ((pd.concat((events_bb, bins_bb), axis=1)['side'] + 1) / 2).astype(int)[-X_test.shape[0]:]

print(f'Primary model accuracy: {accuracy_score(y_test, y_pred_primary)}')
print(f'Primary model precision: {precision_score(y_test, y_pred_primary)}')
print(f'Primary model recall: {recall_score(y_test, y_pred_primary)}')